# StableMotif tutorial 

This is a simple tutorial to demonstrate the basic functionalities of the StableMotif code. This includes:
* Reading in a Boolean model
* Generating the _attractor repertoire_ of the model with basic export tools
* Generating and plotting the _succession diagrams_ with basic plotting and export functions
* Using stable motif control methods


In [ ]:
import StableMotifs as sm
import PyBoolNet
import StableMotifs.Export as ex

## Read in a Boolean model:

In [ ]:
rules='''A*=B
B*=A
C*=A or not D
D*=C
E*=B and F
F*=E'''


In [ ]:
#format the rules to be PyBoolNet compatible 
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
sm.Format.pretty_print_prime_rules({k:primes[k] for k in sorted(primes)})


## Generating the Attractor repertoire

In [ ]:
#explanation of the parameter
max_simulate_size=20

In [ ]:
ar = sm.AttractorRepertoire.from_primes(primes, max_simulate_size=max_simulate_size)

### What do we know about the attractors?

In [ ]:
#Summary of the attractors:
ar.summary()

If we want the attractors in a DataFrame:

In [ ]:
df=ex.attractor_dataframe(ar)
df

To access the attractors as the dictionaries:

In [ ]:
for a in ar.attractors:
    print(a.attractor_dict)

## Generating and plotting the Suuccession Diagrams

If we want to add the attractors as nodes of the succession diagram connected to the terminal nodes of the succession diagram we set:


In [ ]:
include_attractors_in_diagram=True

### Reduced network based succession diagram

In [ ]:
GR=ex.networkx_succession_diagram_reduced_network_based(ar,include_attractors_in_diagram=include_attractors_in_diagram)

In [ ]:
for n in GR.nodes(data=True):
    print(n)

We can plot the succession diagram using matplotlib and networkx. The following parameters can be adjusted:
* __G__ -- (networkx DiGraph) of the succession diagram <br>
* __fig_dimensions__ -- ((float, float)) the width/height dimensions in inches of the figure. By default it's propotional to the number of nodes in the succession diagram <br>
* __pos__ (dict) -- the layout of dictionary of nodes (key) and coordinate tuples (value). By default the "pydot" hierarchical layout is generated by the function
* __detailed_labels__ -- (bool) -- if True (default) the plot will show the corresponding node states to each succession node and attractor. If False the labels are generated based on the motif reduction unique IDs. In this case the attractors are marked with an "A" prefix. 
* __node_size__= (int or list of ints) -- if int, all nodes are of the same size (nr of pixels), if iterable, the nodes in the corresponding order (as returned by G.nodes()) will be sized accordingly. The number of ints in the list has to be equal to the number of nodes. Default is 50* the number of nodes 
* __node_color__ -- (str or list of str) default='grey'. Same system as node_size, all colors codes compatible with matplotlib work.
* __font_size__= (int) default=12
* __font_color__=(str or list of str) default='black'. Same system as node_size, all colors codes compatible with matplotlib work.

In [ ]:
#pos= nx.layout.kamada_kawai_layout(GR)

In [ ]:
ex.plot_nx_succession_diagram(GR)

### Motif based succession diagram

In [ ]:
GM=ex.networkx_succession_diagram_motif_based(ar,include_attractors_in_diagram=True)

In [ ]:
for n in GM.nodes(data=True):
    print(n)

In [ ]:
ex.plot_nx_succession_diagram(GM)

In case we only need the set of unique stable motifs:

In [ ]:
ex.get_motif_set(ar)

### Plotting with yED

Plotting the succession diagrams in matplolib is an quick and efficient way of having a glimpse at the succession diagram, however in the case of large and more complex diagrams this can become inefficient. We suggest exporting the diagrams and plotting them with tools such as yED. Here we explain how to do it:
* first, we export the succession diagram into graphml format. The attributes such as the label are preserved by the format

In [ ]:
ex.save_to_graphml(GR,model_name='succ_diagram_example')

* next, open the saved graphml in yED
* go to Edit -> Properties Mapper
* (optional) in the top left corner of the pop-up window click *import additional configuration* and select the *succession_diagram_yED_properties.cnfx* config file from GitHub. 
* Set up the configuration and click *Apply*
* Finally, we suggest a hierarchical layout. To get this go to Layout -> Hierarchical

## Controlling Attractors

There are several different algorithms and heuristics that can help determine what is the minimum set of nonredundant nodes that need to be fixed in order to make the system reach a desired state _from any other state_. The target state can be a subset of the nodes as well.

The _reprogram_trap_spaces()_ function handles all the implemented control methods. Its inputs are the target node states and the method combinations. Here we give and example for the default _history + internal_ method. Please consult the documentation for the other methods.  

_history + internal_ :
Finds all shortest stable motif histories that result in the target node states being logically fixed. Each stable motif is searched for internal driver nodes. The resulting internal drivers are combined into a single  control set. The returned list consists of all such control sets for all stable motif histories. Each control set eventually becomes self-sustaining.



In [ ]:
target_state={'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1, 'F': 1}
ar.succession_diagram.reprogram_to_trap_spaces(logically_fixed=target_state,
                                               target_method='history',
                                               driver_method='internal')

Please consult the function docstring for more information (by pressing Shift+Tab when the cursor is in the function or by running ar.succession_diagram.reprogram_to_trap_spaces?)
